# Trader

## Real

1 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

2 - Solicita as klines correspondentes

3 - Envia as klines e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold', aciona o 'order_handler'

5 - Dorme

6 - 'Refresca' as klines

7 - Repete a partir do passo 3

## Backing Test

1 - Pede todas as klines

2 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

3 - Envia a fatia (suficientemente longa) de klines mais antiga e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold' procede a 'ordem fictícia'

5 - 'Avança uma' kline, 'refrescando' a fatia

6 - Repete a partir do passo 3

## Observações

1 - É a estratégia que instancia o stop  
2 - Existe um problema, ainda sem causa descoberta: o módulo 'klines_handler' está retornando uma kline a menos  
    
    2.1 - Solução sugerida:  
    
    Em OneMinuteCandlesAmount.total(self, out_candle_number), 'pedir um out_candle_number a mais, truncando em 
    'get_latest' para o número de candles desejado

In [1]:
from modules.assets_handler import AssetsHandler
from pydoc import locate
import time

##Database credentials
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'pandazord_database'                      
DB_USER = 'pandazord'
DB_PASSWORD = 'QBBV9E%pcYKHUcjj'

## Market and Asset
exchange_name = 'binance'
asset_symbol = 'BTCUSDT'

In [2]:
Asset = AssetsHandler(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME, exchange_name, asset_symbol)
op_param = Asset.info().iloc[0].operational_parameters

In [42]:
strategy_name = op_param['strategy']['name']
stop_name = op_param['stop_loss']['name']

Strategy = locate('modules.strategies.' + strategy_name)(op_param)
StopLoss = locate('modules.stop_loss.' + stop_name)(op_param)
OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() 
                       + 'FromDb')(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)

In [38]:
# Ação do trader por ciclo (real)

#while true

sleep_time = op_param['sleep_duration']
is_positioned = bool(Asset.info().position.iloc[0]['side'] != 'closed')

if (is_positioned):
    
    sleep_time = 65
    n_candles = StopLoss.how_many_candles() #TODO: Como será um ciclo, pode ir pra fora do while/for
    klines = KlinesHandler.get_latest(asset_symbol, 
                                      candle_interval='1m', 
                                      number_of_candles=n_candles)
    
    stop = StopLoss.verify(klines, Asset.info())
    
    if (stop.is_true):
        
        order = OrderHandler.execute(stop)
        time.sleep(10)
        order_confirmation = OrderHandler.check(order)
        Asset.update_position(order_confirmation)

else: pass

n_candles = Strategy.how_many_candles() #TODO: Como será um ciclo, pode ir pra fora do while/for
klines = KlinesHandler.get_latest(asset_symbol, 
                                  candle_interval=op_param['candle_interval'], 
                                  number_of_candles=n_candles)

side, leverage = Strategy.what_side_and_leverage(klines)
trade = inspect(side, leverage, Asset.info())

if (trade.is_true):

    order = OrderHandler.execute(trade)
    time.sleep(10)
    order_confirmation = OrderHandler.check(order)
    Asset.update_position(order_confirmation)

time.sleep(sleep_time)

In [11]:
type(op_param['sleep_duration'])

int

In [34]:
is_positioned = bool(Asset.info().position.iloc[0]['side'] != 'closed')

In [35]:
is_positioned

True

In [40]:
Asset.info()

,asset_symbol,auto_update,status,last_updated_by_pid,collect_data_since,trade_on,available_amount,operational_parameters,position
0,BTCUSDT,ON,full,362,2019-02-26 00:00:00+00:00,N,0.0,"{'strategy': {'name': 'CrossSMA', 'parameters'...","{'side': 'closed', 'size': 0.0, 'target_price'..."


In [10]:
max(3,10)

10

In [39]:
stop

NameError: name 'stop' is not defined

In [43]:
op_param

{'strategy': {'name': 'CrossSMA', 'parameters': {'number_samples': [3, 100]}},
 'stop_loss': {'name': 'Default',
  'parameters': {'first_trigger': [1, 3, 10], 'second_trigger': [1, 16, 50]}},
 'price_source': 'ohlc4',
 'sleep_duration': 65,
 'candle_interval': '12h'}

# Deprecated

In [ ]:
import pandas as pd

class RealTradingKlines:
    
    def __init__(self, KlineHandler, asset_symbol, candle_interval, number_of_candles):
        
        self.asset_symbol = asset_symbol
        self.candle_interval = candle_interval
        self.number_of_candles = number_of_candles
        self.klines = pd.DataFrame()
        self.KlinesHandler = KlinesHandler
        
    def update(self):
        
        if (len(self.klines) == 0):
            
            self.klines = self.KlinesHandler.get_latest(self.asset_symbol,
                                                        self.candle_interval,
                                                        self.number_of_candles)
        else:
            
            new_line = self.KlinesHandler.get_latest(self.asset_symbol, 
                                                     self.candle_interval,
                                                     1)
            
            self.klines = self.klines.append(new_line, ignore_index=True)
        
        return self.klines

In [ ]:
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() 
                       + 'FromDb')(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)

Klines = RealTradingKlines(KlinesHandler, asset_symbol, '1m', 100)

In [ ]:
Klines.update()

In [ ]:
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() 
                       + 'FromDb')(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)

In [ ]:
base_klines = KlinesHandler.get_latest(asset_symbol, '1m', 100)

In [ ]:
base_klines.iloc[len(base_klines) - 1]

In [ ]:
base_klines

In [ ]:
last_klines = KlinesHandler.get_latest(asset_symbol, '1m', 1) 

In [ ]:
last_klines.iloc[0].open_time

In [ ]:
from datetime import datetime

number_of_new_samples = (((datetime.utcnow() - last_klines.iloc[0].open_time)).components.minutes)

In [ ]:
number_of_new_samples

In [ ]:
last_klines = KlinesHandler.get_latest(asset_symbol, '1m', number_of_new_samples) 

In [ ]:
last_klines

In [ ]:
base_klines.iloc[len(base_klines) - 1].equals(last_klines.iloc[0])